# Covid-19 Dashboard

##### visualize covid19 information using Tableau

## Abstract

Tableau를 사용한 가시화 프로젝트의 한 예로써 Covid-19 Dashboard에 대하여 실습을 하고자 한다. 코로나 확진자 정보는 매일 업데이트되기에 이를 interactive dashboard를 통해 가시화하는 것은 빠른 시간에 데이터의 동향을 이해하는데 매우 유용하다. 이 노트북에서는 Nguyen Minh Anh의 Covid-19 Tableau Dashboard 구현을 위한 데이터 전처리를 먼저 다룬다.

* Link to Dashboard: [Tableau Public](https://public.tableau.com/app/profile/nguyen.minh.anh/viz/covid19_15924716772030/Dashboard)

이를 위해, 코로나 팬데믹에 따른 관련 데이터를 가장 잘 운영 중인 존스 홉킨즈 대학의 시스템 과학 및 공학 센터 (CSSE; The Center for Systems Science and Engineering)의 깃허브 데이터 리포지토리로부터 실시간 코로나 데이터를 가져온다.

* Data Source: [CSSE](https://github.com/CSSEGISandData/COVID-19)

코로나 데시보드에서 사용하는 지표 중 감염률(infection rate) 등을 구하기 위해서는 해당 국가의 총 인구수 정보가 필요하며, 이를 위해 2020년 국가별 인구수 Kaggle 데이터를 사용한다. 

* Data Source: [Tanu N Prabhu](https://www.kaggle.com/tanuprabhu/population-by-country-2020)

본 프로젝트에서는 Tableau를 이용한 Dashboard 가시화뿐만 아니라 다양한 데이터 소스 (Github, Kaggle)로부터 얻은 정보를 함께 사용할 때 발생되는 이슈들의 해결 방안에 대해서도 학습한다.


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Read datasets from CSSE github repo
confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recoveries = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

In [ ]:
confirmed.head(5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,9/21/21,9/22/21,9/23/21,9/24/21,9/25/21,9/26/21,9/27/21,9/28/21,9/29/21,9/30/21,10/1/21,10/2/21,10/3/21,10/4/21,10/5/21,10/6/21,10/7/21,10/8/21,10/9/21,10/10/21,10/11/21,10/12/21,10/13/21,10/14/21,10/15/21,10/16/21,10/17/21,10/18/21,10/19/21,10/20/21,10/21/21,10/22/21,10/23/21,10/24/21,10/25/21,10/26/21,10/27/21,10/28/21,10/29/21,10/30/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,5,5,...,154712,154757,154800,154960,154960,154960,155072,155093,155128,155174,155191,155191,155191,155287,155309,155380,155429,155448,155466,155508,155540,155599,155627,155682,155688,155739,155764,155776,155801,155859,155891,155931,155940,155944,156040,156071,156124,156166,156196,156210
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,164276,165096,165864,166690,167354,167893,168188,168782,169462,170131,170778,171327,171794,171794,172618,173190,173723,174168,174643,174968,175163,175664,176172,176667,177108,177536,177971,178188,178804,179463,180029,180623,181252,181696,181960,182610,183282,183873,184340,184887
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,...,201948,202122,202283,202449,202574,202722,202877,203045,203198,203359,203517,203657,203789,203915,204046,204171,204276,204388,204490,204597,204695,204790,204900,205005,205106,205199,205286,205364,205453,205529,205599,205683,205750,205822,205903,205990,206069,206160,206270,206358
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,15140,15153,15156,15167,15167,15167,15189,15192,15209,15222,15222,15222,15222,15267,15271,15284,15288,15291,15291,15291,15307,15307,15314,15326,15338,15338,15338,15367,15369,15382,15382,15404,15404,15404,15425,15425,15462,15505,15516,15516
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,52968,53387,53840,54280,54795,55121,55583,56040,56583,56583,58076,58603,58943,58943,59895,60448,60803,61023,61245,61378,61580,61794,62143,62385,62606,62789,62842,63012,63197,63340,63567,63691,63775,63861,63930,64033,64126,64226,64301,64374


In [ ]:
deaths.head(5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,9/21/21,9/22/21,9/23/21,9/24/21,9/25/21,9/26/21,9/27/21,9/28/21,9/29/21,9/30/21,10/1/21,10/2/21,10/3/21,10/4/21,10/5/21,10/6/21,10/7/21,10/8/21,10/9/21,10/10/21,10/11/21,10/12/21,10/13/21,10/14/21,10/15/21,10/16/21,10/17/21,10/18/21,10/19/21,10/20/21,10/21/21,10/22/21,10/23/21,10/24/21,10/25/21,10/26/21,10/27/21,10/28/21,10/29/21,10/30/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,7199,7199,7199,7199,7199,7199,7200,7201,7204,7204,7206,7206,7206,7212,7214,7220,7221,7221,7221,7225,7228,7230,7234,7238,7238,7238,7243,7246,7247,7247,7249,7252,7253,7255,7260,7262,7266,7268,7269,7272
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2594,2601,2609,2619,2629,2640,2653,2668,2685,2698,2705,2710,2713,2713,2725,2734,2746,2753,2759,2768,2777,2783,2788,2797,2807,2810,2820,2829,2841,2849,2855,2863,2870,2874,2880,2888,2893,2902,2909,2916
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,5725,5739,5748,5758,5767,5777,5786,5797,5805,5812,5815,5819,5822,5826,5831,5838,5843,5846,5850,5853,5855,5859,5862,5864,5867,5870,5872,5873,5875,5878,5881,5883,5886,5890,5894,5899,5904,5907,5913,5918
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1414,1434,1460,1471,1487,1501,1513,1526,1537,1537,1567,1574,1577,1577,1587,1598,1603,1613,1618,1622,1629,1642,1650,1653,1660,1662,1664,1670,1678,1682,1685,1693,1695,1697,1701,1702,1703,1705,1707,1708


In [ ]:
recoveries.head(5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,9/21/21,9/22/21,9/23/21,9/24/21,9/25/21,9/26/21,9/27/21,9/28/21,9/29/21,9/30/21,10/1/21,10/2/21,10/3/21,10/4/21,10/5/21,10/6/21,10/7/21,10/8/21,10/9/21,10/10/21,10/11/21,10/12/21,10/13/21,10/14/21,10/15/21,10/16/21,10/17/21,10/18/21,10/19/21,10/20/21,10/21/21,10/22/21,10/23/21,10/24/21,10/25/21,10/26/21,10/27/21,10/28/21,10/29/21,10/30/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
recoveries[['8/4/21','8/5/21']]

,8/4/21,8/5/21
0,82586,0
1,130314,0
2,118409,0
3,14380,0
4,39582,0
...,...,...
259,54332,0
260,312320,0
261,4251,0
262,189658,0


In [ ]:
for c in range(recoveries.columns.get_loc('8/5/21'),len(recoveries.columns)):
    recoveries.iloc[:,c] = recoveries['8/4/21']

In [ ]:
recoveries.loc[:,'8/4/21':]

,8/4/21,8/5/21,8/6/21,8/7/21,8/8/21,8/9/21,8/10/21,8/11/21,8/12/21,8/13/21,8/14/21,8/15/21,8/16/21,8/17/21,8/18/21,8/19/21,8/20/21,8/21/21,8/22/21,8/23/21,8/24/21,8/25/21,8/26/21,8/27/21,8/28/21,8/29/21,8/30/21,8/31/21,9/1/21,9/2/21,9/3/21,9/4/21,9/5/21,9/6/21,9/7/21,9/8/21,9/9/21,9/10/21,9/11/21,9/12/21,...,9/21/21,9/22/21,9/23/21,9/24/21,9/25/21,9/26/21,9/27/21,9/28/21,9/29/21,9/30/21,10/1/21,10/2/21,10/3/21,10/4/21,10/5/21,10/6/21,10/7/21,10/8/21,10/9/21,10/10/21,10/11/21,10/12/21,10/13/21,10/14/21,10/15/21,10/16/21,10/17/21,10/18/21,10/19/21,10/20/21,10/21/21,10/22/21,10/23/21,10/24/21,10/25/21,10/26/21,10/27/21,10/28/21,10/29/21,10/30/21
0,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,...,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586
1,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,...,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314,130314
2,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,...,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409,118409
3,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,...,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380,14380
4,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,...,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582,39582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,...,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,54332,543

## Wide 형식을 Long 형식으로 변환하기 (Transforming Wide to Long format)
CSSE로부터 얻은 데이터 소스는 와이드 형식이며, 이는 Tableau에서 작업하기에 적절하지 않다. 따라서 데이터 전처리의 중요한 작업은 이들 데이터를 롱 형식으로 변환하는 것이다.

In [ ]:
# Transform wide format to long format
confirmed = pd.melt(confirmed, id_vars=confirmed.columns[:4], value_vars = confirmed.columns[4:], var_name = 'date', value_name = 'confirmed')
deaths = pd.melt(deaths, id_vars=deaths.columns[:4], value_vars = deaths.columns[4:], var_name = 'date', value_name = 'deaths')
recoveries = pd.melt(recoveries, id_vars=recoveries.columns[:4], value_vars = recoveries.columns[4:], var_name = 'date', value_name = 'recoveries')

In [ ]:
confirmed.tail(5)

,Province/State,Country/Region,Lat,Long,date,confirmed
180787,NaN,Vietnam,14.058324,108.277199,10/30/21,915603
180788,NaN,West Bank and Gaza,31.952200,35.233200,10/30/21,452997
180789,NaN,Yemen,15.552727,48.516388,10/30/21,9779
180790,NaN,Zambia,-13.133897,27.849332,10/30/21,209722
180791,NaN,Zimbabwe,-19.015438,29.154857,10/30/21,132926


In [ ]:
deaths.tail(5)

,Province/State,Country/Region,Lat,Long,date,deaths
180787,NaN,Vietnam,14.058324,108.277199,10/30/21,22030
180788,NaN,West Bank and Gaza,31.952200,35.233200,10/30/21,4663
180789,NaN,Yemen,15.552727,48.516388,10/30/21,1880
180790,NaN,Zambia,-13.133897,27.849332,10/30/21,3661
180791,NaN,Zimbabwe,-19.015438,29.154857,10/30/21,4675


In [ ]:
recoveries.tail(5)

,Province/State,Country/Region,Lat,Long,date,recoveries
171067,NaN,Vietnam,14.058324,108.277199,10/30/21,54332
171068,NaN,West Bank and Gaza,31.952200,35.233200,10/30/21,312320
171069,NaN,Yemen,15.552727,48.516388,10/30/21,4251
171070,NaN,Zambia,-13.133897,27.849332,10/30/21,189658
171071,NaN,Zimbabwe,-19.015438,29.154857,10/30/21,82994


## 테이블 결합하기 (Combining tables)
다음 단계는 `confirmed`, `deaths`, `recoveries` 테이블을 하나의 테이블로 결합하여 좀더 편리하게 분석할 수 있도록 한다.

하지만, 캐나다의 데이터에서 하나의 문제가 발생한다. `confirmed`와 `deaths` 테이블은 `Province/State`로 캐나다 데이터를 표현하고 있지만, `recoveries` 테이블은 캐나다 전체 데이터를 하나의 값으로 나타내고 있다.

이러한 충돌은 테이블을 함께 결합할 때 일치하지 않는 join key는 생략될 것이기 때문에 사전에 먼저 고려해야 한다.

In [ ]:
confirmed[confirmed['Country/Region'] == 'Canada'].head()

,Province/State,Country/Region,Lat,Long,date,confirmed
39,Alberta,Canada,53.9333,-116.5765,1/22/20,0
40,British Columbia,Canada,53.7267,-127.6476,1/22/20,0
41,Diamond Princess,Canada,0.0000,0.0000,1/22/20,0
42,Grand Princess,Canada,0.0000,0.0000,1/22/20,0
43,Manitoba,Canada,53.7609,-98.8139,1/22/20,0


In [ ]:
confirmed[(confirmed['Country/Region'] == 'Canada') & (confirmed['date'] == '1/1/21')]

,Province/State,Country/Region,Lat,Long,date,confirmed
96294,Alberta,Canada,53.9333,-116.5765,1/1/21,100428
96295,British Columbia,Canada,53.7267,-127.6476,1/1/21,51990
96296,Diamond Princess,Canada,0.0000,0.0000,1/1/21,0
96297,Grand Princess,Canada,0.0000,0.0000,1/1/21,13
96298,Manitoba,Canada,53.7609,-98.8139,1/1/21,24700
96299,New Brunswick,Canada,46.5653,-66.4619,1/1/21,601
96300,Newfoundland and Labrador,Canada,53.1355,-57.6604,1/1/21,390
96301,Northwest Territories,Canada,64.8255,-124.8457,1/1/21,24
96302,Nova Scotia,Canada,44.6820,-63.7443,1/1/21,1486
96303,Nunavut,Canada,70.2998,-83.1076,1/1/21,266


In [ ]:
confirmed[(confirmed['Country/Region'] == 'Canada') & (confirmed['date'] == '1/1/21')]['confirmed'].sum()

591149

In [ ]:
# Summarize confirmed and deaths data by date
confirmed_canada = confirmed[confirmed['Country/Region'] == 'Canada'].groupby('date').sum()[['confirmed']]
deaths_canada = deaths[deaths['Country/Region'] == 'Canada'].groupby('date').sum()[['deaths']]

In [ ]:
confirmed_canada.loc['1/1/21':'1/1/21']

,confirmed
date,
1/1/21,591149


In [ ]:
# Extract columns from recoveries table
recoveries_canada = recoveries[recoveries['Country/Region'] == 'Canada']
canada_template = recoveries_canada[recoveries_canada.columns[:-1]].reset_index(drop=True)

In [ ]:
recoveries_canada

,Province/State,Country/Region,Lat,Long,date,recoveries
39,NaN,Canada,56.1304,-106.3468,1/22/20,0
303,NaN,Canada,56.1304,-106.3468,1/23/20,0
567,NaN,Canada,56.1304,-106.3468,1/24/20,0
831,NaN,Canada,56.1304,-106.3468,1/25/20,0
1095,NaN,Canada,56.1304,-106.3468,1/26/20,0
...,...,...,...,...,...,...
169791,NaN,Canada,56.1304,-106.3468,10/26/21,1405971
170055,NaN,Canada,56.1304,-106.3468,10/27/21,1405971
170319,NaN,Canada,56.1304,-106.3468,10/28/21,1405971
170583,NaN,Canada,56.1304,-106.3468,10/29/21,1405971


In [ ]:
canada_template

,Province/State,Country/Region,Lat,Long,date
0,NaN,Canada,56.1304,-106.3468,1/22/20
1,NaN,Canada,56.1304,-106.3468,1/23/20
2,NaN,Canada,56.1304,-106.3468,1/24/20
3,NaN,Canada,56.1304,-106.3468,1/25/20
4,NaN,Canada,56.1304,-106.3468,1/26/20
...,...,...,...,...,...
643,NaN,Canada,56.1304,-106.3468,10/26/21
644,NaN,Canada,56.1304,-106.3468,10/27/21
645,NaN,Canada,56.1304,-106.3468,10/28/21
646,NaN,Canada,56.1304,-106.3468,10/29/21


`pd.merge(
    left,
    right,
    how="inner",
    on=None,
    left_on=None,
    right_on=None,
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)`

* `left_on`은 왼쪽 DataFrame으로부터 columns 혹은 index level을 key로 사용한다는 의미이다.
* `right_index = True`는 오른쪽 DataFrame으로부터 index (혹은 row labels)를 join key로 사용한다는 의미이다.

In [ ]:
# Join aggrregated confirmed and deaths data with extracted columns
confirmed_canada = canada_template.merge(confirmed_canada, how='inner', left_on='date', right_index=True)
deaths_canada = canada_template.merge(deaths_canada, how='inner', left_on='date', right_index=True)

In [ ]:
confirmed_canada.head()

,Province/State,Country/Region,Lat,Long,date,confirmed
0,NaN,Canada,56.1304,-106.3468,1/22/20,0
1,NaN,Canada,56.1304,-106.3468,1/23/20,0
2,NaN,Canada,56.1304,-106.3468,1/24/20,0
3,NaN,Canada,56.1304,-106.3468,1/25/20,0
4,NaN,Canada,56.1304,-106.3468,1/26/20,1


In [ ]:
deaths_canada.head()

,Province/State,Country/Region,Lat,Long,date,deaths
0,NaN,Canada,56.1304,-106.3468,1/22/20,0
1,NaN,Canada,56.1304,-106.3468,1/23/20,0
2,NaN,Canada,56.1304,-106.3468,1/24/20,0
3,NaN,Canada,56.1304,-106.3468,1/25/20,0
4,NaN,Canada,56.1304,-106.3468,1/26/20,0


In [ ]:
# Add the agrregated data for Canada back to confirmed and deaths table
confirmed = confirmed[confirmed['Country/Region'] != 'Canada'].append(confirmed_canada)
deaths = deaths[deaths['Country/Region'] != 'Canada'].append(deaths_canada)

In [ ]:
# Join confirmed, deaths and recoveries data together
data = confirmed.merge(deaths, how='inner',on=['Country/Region','Province/State','date']).merge(recoveries, how='inner',on=['Country/Region','Province/State','date'])
data['recoveries'] = data['recoveries'].astype("int")
data = data[['Province/State','Country/Region','date','Lat','Long','confirmed','deaths','recoveries']]

In [ ]:
data.sample(10)

,Province/State,Country/Region,date,Lat,Long,confirmed,deaths,recoveries
140586,NaN,"Korea, South",7/9/21,35.907757,127.766922,166722,2038,153153
136955,NaN,Peru,6/25/21,-9.190000,-75.015200,2043262,191447,2001357
131094,NaN,Greece,6/3/21,39.074200,21.824300,406751,12184,93764
60477,Montserrat,United Kingdom,9/7/20,16.742498,-62.187366,13,1,12
41269,NaN,United Arab Emirates,6/26/20,23.424076,53.847818,46973,310,35469
20280,NaN,Botswana,4/8/20,-22.328500,24.684900,6,1,0
16595,NaN,Bhutan,3/25/20,27.514200,90.433600,2,0,0
152145,NaN,India,8/22/21,20.593684,78.962880,32449306,434756,30974748
61162,NaN,Kuwait,9/10/20,29.311660,47.481766,92822,556,83040
31385,NaN,Denmark,5/20/20,56.263900,9.501800,11117,554,9536


## 인구수 데이터 (Population Data)
코로나-19 데시보드에서 사용하는 값 중에 감염률(infection rate)은 $confirmed / population$으로 구해진다. 각 국가별 인구수는 CSSE 데이터셋에서는 구할 수가 없기 때문에 다른 데이터 소스를 결합할 필요가 있다. 이를 위해 다음의 Kaggle 데이터를 사용한다.

Source: [Tanu N Prabhu](https://www.kaggle.com/tanuprabhu/population-by-country-2020)

Kaggle 데이터를 구하는 과정은 **[프로젝트 02] 세계행복보고서**를 참조하도록 하자.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

In [ ]:
from os import path

if path.exists("/content/gdrive/My Drive/Kaggle"):
    print("Kaggle 디렉토리는 이미 있습니다.")
else:
    %mkdir "/content/gdrive/My Drive/Kaggle"

Kaggle 디렉토리는 이미 있습니다.


In [ ]:
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


이미 Kaggle로 부터 인구수 데이터를 다운로드하였다면 아래의 모듈은 더이상 반복해서 수행하지 않도록 하였습니다.


In [ ]:
# Kaggle로부터 인구수 데이터를 다운로드하는 모듈

def getPopulationFromKaggle():
    # 이 API Command는 해당 사이트에서 직접 복사해옴 
    !kaggle datasets download -d tanuprabhu/population-by-country-2020

    %mkdir input/covid-19
    %mv population-by-country-2020.zip input/covid-19

    %cd input/covid-19

    !unzip \*.zip && rm *.zip

In [ ]:
if not path.exists("input"): # input directory가 없을 경우에만 생성
    %mkdir input
if not path.exists("input/covid-19"): # covid-19 디렉토리가 없는 경우 캐글로부터 데이터를 가져옴
    getPopulationFromKaggle()
else:
    %cd input/covid-19
    print("input/covid-19 디렉토리가 있으므로 인구수 데이터를 다운로드한 것으로 판단함")

/content/gdrive/My Drive/Kaggle/input/covid-19
input/covid-19 디렉토리가 있으므로 인구수 데이터를 다운로드한 것으로 판단함


In [ ]:
# Read dataset
population = pd.read_csv('/content/gdrive/MyDrive/Kaggle/input/covid-19/population_by_country_2020.csv')

In [ ]:
population.sample(10)

,Country (or dependency),Population (2020),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share
63,Kazakhstan,18815231,1.21 %,225280,7,2699700,-18000.0,2.8,31,58 %,0.24 %
2,United States,331341050,0.59 %,1937734,36,9147420,954806.0,1.8,38,83 %,4.25 %
49,Venezuela,28421581,-0.28 %,-79889,32,882050,-653249.0,2.3,30,N.A.,0.36 %
214,Turks and Caicos,38806,1.38 %,526,41,950,NaN,N.A.,N.A.,89 %,0.00 %
208,Greenland,56787,0.17 %,98,0,410450,NaN,N.A.,N.A.,87 %,0.00 %
4,Pakistan,221612785,2.00 %,4327022,287,770880,-233379.0,3.6,23,35 %,2.83 %
187,Samoa,198643,0.67 %,1317,70,2830,-2803.0,3.9,22,18 %,0.00 %
147,North Macedonia,2083359,0.00 %,-85,83,25220,-1000.0,1.5,39,59 %,0.03 %
163,Guyana,787215,0.48 %,3786,4,196850,-6000.0,2.5,27,27 %,0.01 %
29,Spain,46757980,0.04 %,18002,94,498800,40000.0,1.3,45,80 %,0.60 %


여러 소스로부터 구한 데이터를 결합하는 과정에서 항상 발생될 수 있는 문제는 값의 이름이 일치하지 않는 것이다.



In [ ]:
for c in data['Country/Region'].unique():
  if c not in population['Country (or dependency)'].unique():
    print(c)

Burma
Congo (Brazzaville)
Congo (Kinshasa)
Cote d'Ivoire
Czechia
Diamond Princess
Korea, South
Kosovo
MS Zaandam
Saint Kitts and Nevis
Saint Vincent and the Grenadines
Sao Tome and Principe
Summer Olympics 2020
Taiwan*
US
West Bank and Gaza


위에서 처럼 코로나-19에 있는 국가명이 인구수 데이터의 국가명과 일치하지 않거나 없는 국가명이 다소 있으며, 이는 불행히도 수작업으로 직접 수정하여야 한다.

이때 population 데이터에 있는 국가명을 참조하기 위해서 다음 코드를 실행시켜 결과를 확인해보자.

In [ ]:
data[data['Country/Region'] == 'Kosovo']

,Province/State,Country/Region,date,Lat,Long,confirmed,deaths,recoveries
145,NaN,Kosovo,1/22/20,42.602636,20.902977,0,0,0
408,NaN,Kosovo,1/23/20,42.602636,20.902977,0,0,0
671,NaN,Kosovo,1/24/20,42.602636,20.902977,0,0,0
934,NaN,Kosovo,1/25/20,42.602636,20.902977,0,0,0
1197,NaN,Kosovo,1/26/20,42.602636,20.902977,0,0,0
...,...,...,...,...,...,...,...,...
169254,NaN,Kosovo,10/26/21,42.602636,20.902977,160658,2975,105688
169517,NaN,Kosovo,10/27/21,42.602636,20.902977,160680,2976,105688
169780,NaN,Kosovo,10/28/21,42.602636,20.902977,160696,2976,105688
170043,NaN,Kosovo,10/29/21,42.602636,20.902977,160724,2976,105688


In [ ]:
for c in population['Country (or dependency)'].unique():
  if c not in data['Country/Region'].unique():
    print(c)

United States
DR Congo
Myanmar
South Korea
Côte d'Ivoire
North Korea
Taiwan
Czech Republic (Czechia)
Hong Kong
Turkmenistan
Congo
State of Palestine
Puerto Rico
Réunion
Macao
Western Sahara
Guadeloupe
Martinique
French Guiana
New Caledonia
French Polynesia
Mayotte
Sao Tome & Principe
Channel Islands
Guam
Curaçao
St. Vincent & Grenadines
Aruba
Tonga
U.S. Virgin Islands
Isle of Man
Cayman Islands
Bermuda
Northern Mariana Islands
Greenland
American Samoa
Saint Kitts & Nevis
Faeroe Islands
Sint Maarten
Turks and Caicos
Saint Martin
Gibraltar
British Virgin Islands
Caribbean Netherlands
Cook Islands
Anguilla
Tuvalu
Wallis & Futuna
Nauru
Saint Barthelemy
Saint Helena
Saint Pierre & Miquelon
Montserrat
Falkland Islands
Niue
Tokelau


In [ ]:
country_mapper = {
    'Burma': 'Myanmar',
    'Congo (Brazzaville)': 'Congo',
    'Congo (Kinshasa)': 'Congo',
    "Cote d'Ivoire": "Côte d'Ivoire",
    'Czechia': 'Czech Republic (Czechia)',
    'Korea, South': 'South Korea',
    'Saint Vincent and the Grenadines': 'St. Vincent & Grenadines',
    'Saint Kitts and Nevis': 'Saint Kitts & Nevis',
    'Sao Tome and Principe': 'Sao Tome & Principe',
    'Taiwan*': 'Taiwan',
    'US': 'United States',
    'West Bank and Gaza': 'Israel'
}
data['Country/Region'] = data['Country/Region'].replace(country_mapper)
data.index = data['Country/Region']

위에서 맵핑이 어려운 국가명은 Diamond Princess, Kosovo, MS Zaandam, Summer Olympics 2020으로 Diamond Princess, MS Zaandam은 유람선 내의 코로나 환자수 상황을 위한 것이고, Summer Olympics 2020 역시 올림픽 행사를 위해 임시로 만든 것이므로 이번 프로젝트에서는 다루지 않는 것으로 한다. 그리고 Kosovo의 인구수 (1,804,376)는 Seribia의 인구수(6,926,705)에 포함되어 있기에 이를 분리할 필요가 있다. *이 부분은 추후 다시 확인하도록 한다.*

In [ ]:
population[population['Country (or dependency)'] == 'Serbia']

,Country (or dependency),Population (2020),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share
98,Serbia,8731081,-0.40 %,-34864,100,87460,4000.0,1.5,42,56 %,0.11 %


In [ ]:
data[data['Country/Region'] == 'Kosovo']

,Province/State,Country/Region,date,Lat,Long,confirmed,deaths,recoveries
Country/Region,,,,,,,,
Kosovo,NaN,Kosovo,1/22/20,42.602636,20.902977,0,0,0
Kosovo,NaN,Kosovo,1/23/20,42.602636,20.902977,0,0,0
Kosovo,NaN,Kosovo,1/24/20,42.602636,20.902977,0,0,0
Kosovo,NaN,Kosovo,1/25/20,42.602636,20.902977,0,0,0
Kosovo,NaN,Kosovo,1/26/20,42.602636,20.902977,0,0,0
...,...,...,...,...,...,...,...,...
Kosovo,NaN,Kosovo,10/26/21,42.602636,20.902977,160658,2975,105688
Kosovo,NaN,Kosovo,10/27/21,42.602636,20.902977,160680,2976,105688
Kosovo,NaN,Kosovo,10/28/21,42.602636,20.902977,160696,2976,105688


In [ ]:
# Export data
data.to_csv('covid19.csv')

## Tableau 데시보드 만들기

이제 잘 다듬어진 데이터를 이용하여 데시보드를 만들어보자.


#Dashboard Link: https://public.tableau.com/app/profile/jeongsik.park/viz/20181141391/Dashboard